## Hyperparameter optimization using hyperopt

In [8]:
from __future__ import print_function, division, absolute_import
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))
# from clr_callback import CyclicLR
# import dill
from AudioDataGenerator import AudioDataGenerator
import os
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
import pandas as pd
import tables
from datetime import datetime
import argparse
from keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Activation, AveragePooling1D
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.optimizers import Adam  # Nadam, Adamax
from keras.callbacks import TensorBoard, Callback, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger
from keras import backend as K
from keras.utils import plot_model
from custom_layers import Conv1D_zerophase_linear, Conv1D_linearphase, Conv1D_zerophase,\
    DCT1D, Conv1D_gammatone, Conv1D_linearphaseType
from heartnet_v1 import log_macc, write_meta, compute_weight, reshape_folds, results_log
from utils import DenseNet
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from keras.losses import categorical_crossentropy
sns.set()
import ast # for list import

In [9]:
def branch(input_tensor,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable):

    num_filt1, num_filt2 = num_filt
    t = Conv1D(num_filt1, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = MaxPooling1D(pool_size=subsam)(t)
    t = Conv1D(num_filt2, kernel_size=kernel_size,
               kernel_initializer=initializers.he_normal(seed=random_seed),
               padding=padding,
               use_bias=bias,
               trainable=trainable,
               kernel_constraint=max_norm(maxnorm),
               kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = MaxPooling1D(pool_size=subsam)(t)
    # t = Flatten()(t)
    return t

In [10]:
foldname='fold0_noFIR'


In [11]:
random_seed=1
batch_size=64
fold_dir = '/media/taufiq/Data1/heart_sound/feature/segmented_noFIR/folds_dec_2018/'
log_dir= '/media/taufiq/Data/heart_sound/Heart_Sound/codes/logs/'
bn_momentum = 0.99
eps= 1.1e-5
bias=False
l2_reg=0.
# l2_reg_dense=0.
kernel_size=5
maxnorm=10000.
dropout_rate=0.5
dropout_rate_dense=0.
padding='valid'
activation_function='relu'
subsam=2
load_path=False
lr=0.0012843784 
lr_decay=0.0001132885
FIR_train=True
num_filt=(8,4)
num_dense=20
params = dict()
print("values initialized")

model_dir = '/media/taufiq/Data1/heart_sound/models/'
fold_dir = '/media/taufiq/Data1/heart_sound/feature/segmented_noFIR/folds_dec_2018/'
log_name = foldname + ' ' + str(datetime.now())
log_dir = '/media/taufiq/Data1/heart_sound/logs/'
if not os.path.exists(model_dir + log_name):
    os.makedirs(model_dir + log_name)
checkpoint_name = model_dir + log_name + "/" + 'weights.{epoch:04d}-{val_acc:.4f}.hdf5'
results_path = '/media/taufiq/Data1/heart_sound/results_2class.csv'

num_filt = (8, 4)
num_dense = 20

bn_momentum = 0.99
eps = 1.1e-5
bias = False
l2_reg = 0.04864911065093751
l2_reg_dense = 0.
kernel_size = 5
maxnorm = 10000.
dropout_rate = 0.5
dropout_rate_dense = 0.
padding = 'valid'
activation_function = 'relu'
subsam = 2
FIR_train= True
trainable = True
decision = 'majority'  # Decision algorithm for inference over total recording ('majority','confidence')

# lr =  0.00125 ## After bayesian optimization

###### lr_decay optimization ######
lr_decay =0.0001132885

values initialized


In [12]:
feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[:]
val_parts = feat.root.val_parts[0, :]

############## Relabeling ################

for i in range(0, y_train.shape[0]):
    if y_train[i] == -1:
        y_train[i] = 0  ## Label 0 for normal 1 for abnormal
for i in range(0, y_val.shape[0]):
    if y_val[i] == -1:
        y_val[i] = 0

############# Parse Database names ########

train_files = []
for each in feat.root.train_files[:][0]:
    train_files.append(chr(each))
print(len(train_files))
val_files = []
for each in feat.root.val_files[:][0]:
    val_files.append(chr(each))
print(len(val_files))

################### Reshaping ############

x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

79810
6710
(79810, 2500, 1)
(79810, 1)
(6710, 2500, 1)
(6710, 1)


In [13]:
datagen = AudioDataGenerator(shift=.1,
                             # roll_range=.1,
                             # fill_mode='reflect',
                             # featurewise_center=True,
                             # zoom_range=.1,
                             # zca_whitening=True,
                             # samplewise_center=True,
                             # samplewise_std_normalization=True,
                             )

In [17]:
def heartnet(load_path,activation_function='relu', bn_momentum=0.99,
             bias=False, dropout_rate=0.5, dropout_rate_dense=0.0,
             eps=1.1e-5, kernel_size=5, l2_reg=0.0, l2_reg_dense=0.0,
             lr=0.0012843784, lr_decay=0.0001132885, maxnorm=10000.,
             padding='valid', random_seed=1, subsam=2, num_filt=(8, 4),
             num_dense=20,FIR_train=False,trainable=True,type_=1,args=1):

    input = Input(shape=(2500, 1))

    coeff_path = '/media/taufiq/Data1/heart_sound/heartnetTransfer/filterbankcoeff60.mat'
    coeff = tables.open_file(coeff_path)
    b1 = coeff.root.b1[:]
    b1 = np.hstack(b1)
    b1 = np.reshape(b1, [b1.shape[0], 1, 1])

    b2 = coeff.root.b2[:]
    b2 = np.hstack(b2)
    b2 = np.reshape(b2, [b2.shape[0], 1, 1])

    b3 = coeff.root.b3[:]
    b3 = np.hstack(b3)
    b3 = np.reshape(b3, [b3.shape[0], 1, 1])

    b4 = coeff.root.b4[:]
    b4 = np.hstack(b4)
    b4 = np.reshape(b4, [b4.shape[0], 1, 1])

    ## Conv1D_linearphase

    # input1 = Conv1D_linearphase(1 ,61, use_bias=False,
    #                 # kernel_initializer=initializers.he_normal(random_seed),
    #                 weights=[b1[30:]],
    #                 padding='same',trainable=FIR_train)(input)
    # input2 = Conv1D_linearphase(1, 61, use_bias=False,
    #                 # kernel_initializer=initializers.he_normal(random_seed),
    #                 weights=[b2[30:]],
    #                 padding='same',trainable=FIR_train)(input)
    # input3 = Conv1D_linearphase(1, 61, use_bias=False,
    #                 # kernel_initializer=initializers.he_normal(random_seed),
    #                 weights=[b3[30:]],
    #                 padding='same',trainable=FIR_train)(input)
    # input4 = Conv1D_linearphase(1, 61, use_bias=False,
    #                 # kernel_initializer=initializers.he_normal(random_seed),
    #                 weights=[b4[30:]],
    #                 padding='same',trainable=FIR_train)(input)

    ## Conv1D_linearphase Anti-Symmetric
    #

    if type_ % 2:
        weight_idx = 30
    else:
        weight_idx = 31

    input1 = Conv1D_linearphaseType(1 ,61, use_bias=False,
                    # kernel_initializer=initializers.he_normal(random_seed),
                    weights=[b1[weight_idx:]],
                    padding='same',trainable=FIR_train, type = type_)(input)
    input2 = Conv1D_linearphaseType(1, 61, use_bias=False,
                    # kernel_initializer=initializers.he_normal(random_seed),
                    weights=[b2[weight_idx:]],
                    padding='same',trainable=FIR_train, type = type_)(input)
    input3 = Conv1D_linearphaseType(1, 61, use_bias=False,
                    # kernel_initializer=initializers.he_normal(random_seed),
                    weights=[b3[weight_idx:]],
                    padding='same',trainable=FIR_train, type = type_)(input)
    input4 = Conv1D_linearphaseType(1, 61, use_bias=False,
                    # kernel_initializer=initializers.he_normal(random_seed),
                    weights=[b4[weight_idx:]],
                    padding='same',trainable=FIR_train, type = type_)(input)

    #Conv1D_gammatone

    # input1 = Conv1D_gammatone(kernel_size=81,filters=1,fsHz=1000,use_bias=False,padding='same')(input)
    # input2 = Conv1D_gammatone(kernel_size=81,filters=1,fsHz=1000,use_bias=False,padding='same')(input)
    # input3 = Conv1D_gammatone(kernel_size=81,filters=1,fsHz=1000,use_bias=False,padding='same')(input)
    # input4 = Conv1D_gammatone(kernel_size=81,filters=1,fsHz=1000,use_bias=False,padding='same')(input)

    t1 = branch(input1,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t2 = branch(input2,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t3 = branch(input3,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    t4 = branch(input4,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

    merged = Concatenate(axis=-1)([t1, t2, t3, t4])
    merged = DenseNet(merged,
                      depth=int(3*args[0]+4),
                      nb_dense_block=args[1],
                      growth_rate=int(args[2]),
#                       kernel_size=args[3],
                      kernel_size=5,
#                       nb_filter=args[4],
                      nb_filter=16,
#                       dropout_rate=args[5],
                      dropout_rate=dropout_rate,
                     )
    # 7,4,4,5,16; 7,1,4,5,16

    # merged = DCT1D()(merged)
    merged = Flatten()(merged)
    merged = Dense(num_dense,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense))(merged)
    # merged = BatchNormalization(epsilon=eps,momentum=bn_momentum,axis=-1) (merged)
    # merged = Activation(activation_function)(merged)
    merged = Dropout(rate=dropout_rate_dense, seed=random_seed)(merged)
    merged = Dense(2, activation='softmax')(merged)

    model = Model(inputs=input, outputs=merged)

    if load_path:  # If path for loading model was specified
        model.load_weights(filepath=load_path, by_name=False)

    adam = Adam(lr=lr, decay=lr_decay)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
def objective(args):
    print("args {}".format(args))
    
    log_name = "hyperopt-{}".format(args) + str(datetime.now())
    if not os.path.exists(model_dir + log_name):
        os.makedirs(model_dir + log_name)
    checkpoint_name = model_dir + log_name + "/" + 'weights.{epoch:04d}-{val_acc:.4f}.hdf5'
    
    
    csv_logger = CSVLogger(log_dir + log_name + '/training.csv')
    tensbd = TensorBoard(log_dir=log_dir + log_name,
                            batch_size=batch_size)
    modelcheckpnt = ModelCheckpoint(filepath=checkpoint_name,
                                monitor='val_acc',
                                save_best_only=True, mode='max')
    
    model = heartnet(load_path,activation_function, bn_momentum, bias, dropout_rate, dropout_rate_dense,
                         eps, kernel_size, l2_reg, l2_reg_dense, lr, lr_decay, maxnorm,
                         padding, random_seed, subsam, num_filt, num_dense, FIR_train, trainable, 1, args)
    model.summary()
    model.fit_generator(datagen.flow(x_train, y_train, batch_size, shuffle=True,
                                     seed=random_seed),
                            steps_per_epoch=len(x_train) // batch_size,
                            # max_queue_size=20,
                            use_multiprocessing=False,
                            epochs=20,
                            verbose=1,
                            shuffle=True,
                            callbacks=[modelcheckpnt,
                                       log_macc(val_parts, decision=decision,verbose=1, val_files=val_files),
                                       tensbd, csv_logger],
                            validation_data=(x_val, y_val),
                            initial_epoch=0,
                            )
    y_pred = model.predict(x_val)
    loss = K.eval(K.mean(K.variable(K.eval(categorical_crossentropy(K.variable(y_val),K.variable(y_pred))))))
    params[str(args)]=loss
    print("Loss: %d" % loss)
    return loss    

In [ ]:
from hyperopt import hp, Trials, fmin, tpe
trials = Trials()
best = fmin(objective,
    space=[hp.quniform('depth',1,30,1),
           hp.choice('num_block',[1,2,3]),
           hp.quniform('growth',1,20,1),
#            hp.choice('kernel_size',[3,5,7]),
#            hp.choice('num_filters',[8,16,32]),
#            hp.normal('dropout',0.4,0.1)
          ],
    algo=tpe.suggest,
    max_evals=20,
    trials=trials)

args (23.0, 1, 4.0)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
conv1d_linearphase_type_5 (Conv (None, 2500, 1)      31          input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_6 (Conv (None, 2500, 1)      31          input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_type_7 (Conv (None, 2500, 1)      31          input_2[0][0]                    
_________________________________________________________________________________________

Epoch 1/20
6710/6710 [==============================] - 5s 721us/step
TN:96,FP:50,FN:50,TP:88,Macc:0.647607652813,F1:0.637675608638
Epoch 2/20
6710/6710 [==============================] - 3s 513us/step
TN:72,FP:74,FN:27,TP:111,Macc:0.648749204874,F1:0.68730107123
Epoch 3/20
6710/6710 [==============================] - 3s 510us/step
TN:57,FP:89,FN:14,TP:124,Macc:0.644480791252,F1:0.70654741342
Epoch 4/20
6710/6710 [==============================] - 4s 523us/step
TN:47,FP:99,FN:4,TP:134,Macc:0.64646609966,F1:0.722366785486
Epoch 5/20
6710/6710 [==============================] - 3s 510us/step
TN:39,FP:107,FN:0,TP:138,Macc:0.633561593918,F1:0.720621519768
Epoch 6/20
6710/6710 [==============================] - 3s 513us/step
TN:67,FP:79,FN:24,TP:114,Macc:0.642495482844,F1:0.688816358381
Epoch 7/20
6710/6710 [==============================] - 4s 525us/step
TN:45,FP:101,FN:4,TP:134,Macc:0.639616785107,F1:0.718493489129
Epoch 8/20
6710/6710 [==============================] - 4s 526us/step
TN:3

Epoch 1/20
6710/6710 [==============================] - 7s 1ms/step
TN:37,FP:109,FN:7,TP:131,Macc:0.601349962546,F1:0.693116553297
Epoch 2/20
6710/6710 [==============================] - 5s 718us/step
TN:49,FP:97,FN:12,TP:126,Macc:0.624329909276,F1:0.698055704246
Epoch 3/20
6710/6710 [==============================] - 5s 715us/step
TN:27,FP:119,FN:2,TP:136,Macc:0.585219330369,F1:0.692106908051
Epoch 4/20
6710/6710 [==============================] - 5s 740us/step
TN:23,FP:123,FN:0,TP:138,Macc:0.578767077498,F1:0.691724307873
Epoch 5/20
6710/6710 [==============================] - 5s 729us/step
TN:28,FP:118,FN:2,TP:136,Macc:0.588643987645,F1:0.693872493737
Epoch 6/20
6710/6710 [==============================] - 5s 709us/step
TN:45,FP:101,FN:5,TP:133,Macc:0.63599359699,F1:0.715048587475
Epoch 7/20
6710/6710 [==============================] - 5s 724us/step
TN:36,FP:110,FN:2,TP:136,Macc:0.616041245855,F1:0.708328227846
Epoch 8/20
6710/6710 [==============================] - 5s 739us/step
TN

Epoch 1/20
6710/6710 [==============================] - 10s 1ms/step
TN:146,FP:0,FN:138,TP:0,Macc:0.499999962329,F1:0.0
Epoch 2/20
6710/6710 [==============================] - 7s 1ms/step
TN:146,FP:0,FN:138,TP:0,Macc:0.499999962329,F1:0.0
Epoch 3/20
6710/6710 [==============================] - 7s 1ms/step
TN:68,FP:78,FN:26,TP:112,Macc:0.638673763886,F1:0.68292142174
Epoch 4/20
6710/6710 [==============================] - 7s 1ms/step
TN:77,FP:69,FN:26,TP:112,Macc:0.669495679372,F1:0.702188908916
Epoch 5/20
6710/6710 [==============================] - 7s 1ms/step
TN:73,FP:73,FN:32,TP:106,Macc:0.634057921566,F1:0.668764261724
Epoch 6/20
6710/6710 [==============================] - 7s 1ms/step
TN:32,FP:114,FN:12,TP:126,Macc:0.56611073558,F1:0.666661528384
Epoch 7/20
6710/6710 [==============================] - 7s 1ms/step
TN:43,FP:103,FN:10,TP:128,Macc:0.611028341853,F1:0.693761745707
Epoch 8/20
6710/6710 [==============================] - 7s 1ms/step
TN:82,FP:64,FN:36,TP:102,Macc:0.650387

Epoch 1/20
6710/6710 [==============================] - 15s 2ms/step
TN:30,FP:116,FN:4,TP:134,Macc:0.588246925964,F1:0.690716568641
Epoch 2/20
6710/6710 [==============================] - 10s 1ms/step
TN:60,FP:86,FN:16,TP:122,Macc:0.647508386847,F1:0.705196992455
Epoch 3/20
6710/6710 [==============================] - 10s 1ms/step
TN:37,FP:109,FN:6,TP:132,Macc:0.604973150663,F1:0.696564786665
Epoch 4/20
6710/6710 [==============================] - 10s 1ms/step
TN:36,FP:110,FN:1,TP:137,Macc:0.619664433972,F1:0.71168321191
Epoch 5/20
6710/6710 [==============================] - 10s 1ms/step
TN:119,FP:27,FN:68,TP:70,Macc:0.66115738406,F1:0.595739292542
Epoch 6/20
6710/6710 [==============================] - 10s 1ms/step
TN:142,FP:4,FN:118,TP:20,Macc:0.558765095564,F1:0.24691077034
Epoch 7/20
6710/6710 [==============================] - 10s 1ms/step
TN:139,FP:7,FN:113,TP:25,Macc:0.56660706432,F1:0.294114247374
Epoch 8/20
6710/6710 [==============================] - 10s 1ms/step
TN:112,FP:3

In [ ]:
import matplotlib.pyplot as plt

D =(params)

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()),rotation=90)
plt.yscale('log',nonposy='clip')
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()